# Dummy Column

Using dask futures API to pass cuDF column and Series objects between workers. Columns work but Series do not.

In [1]:
import os

In [2]:
base_env = {
    "NOTEBOOK": "1",
    "UCX_RNDV_SCHEME": "put_zcopy",
    "UCX_MEMTYPE_CACHE": "n",
    "UCX_TLS": "rc,cuda_copy,cuda_ipc",
    "CUDA_VISIBLE_DEVICES": "4,5",
}
os.environ.update(base_env)

In [3]:
import dask
import distributed

In [4]:
print(f'Dask: {dask.__file__}')
print(f'Distributed: {distributed.__file__}')

Dask: /home/nfs/bzaitlen/GitRepos/dask/dask/__init__.py
Distributed: /home/nfs/bzaitlen/GitRepos/distributed/distributed/__init__.py


In [5]:
from dask.distributed import Client, wait
# from dask_cuda import DGX

#cluster = DGX(CUDA_VISIBLE_DEVICES=[2,3], 
#              dashboard_address='10.33.227.165:8789')
#client = Client(cluster)
client = Client('ucx://10.33.225.165:8986')
client

Client Scheduler: ucx://10.33.225.165:8986 Dashboard: http://10.33.225.165:8787/status,Cluster Workers: 2 Cores: 2 Memory: 64.00 GB


In [6]:
import sys, os

In [7]:
path = os.getcwd()

In [8]:
# You can add it to your global config with the following yaml
#     distributed:
#       worker:
#         preload:
#           - dask_cuda.initialize_context
def set_nb_context():
    import numba.cuda
    try:
        numba.cuda.current_context()
    except Exception:
        print("FAILED EXCEPTION!")

In [9]:
client.run(set_nb_context)

{'ucx://10.33.225.185:41973': None, 'ucx://10.33.225.185:50389': None}

In [10]:
def set_os(path):
    import sys
    sys.path.append(path)
    return sys.path

In [11]:
client.run(set_os, path)

{'ucx://10.33.225.185:41973': ['/home/nfs/bzaitlen/worker-ybgzhgnf',
  '/home/nfs/bzaitlen/miniconda3/envs/cudf-dev/bin',
  '/home/nfs/bzaitlen/miniconda3/envs/cudf-dev/lib/python37.zip',
  '/home/nfs/bzaitlen/miniconda3/envs/cudf-dev/lib/python3.7',
  '/home/nfs/bzaitlen/miniconda3/envs/cudf-dev/lib/python3.7/lib-dynload',
  '/home/nfs/bzaitlen/.local/lib/python3.7/site-packages',
  '/home/nfs/bzaitlen/miniconda3/envs/cudf-dev/lib/python3.7/site-packages',
  '/home/nfs/bzaitlen/miniconda3/envs/cudf-dev/lib/python3.7/site-packages/nvstrings_cudaunknown-0.0.0.dev0-py3.7-linux-x86_64.egg',
  '/home/nfs/bzaitlen/GitRepos/cudf/python/cudf',
  '/home/nfs/bzaitlen/GitRepos/cudf/python/dask_cudf',
  '/home/nfs/bzaitlen/GitRepos/distributed',
  '/home/nfs/bzaitlen/GitRepos/dask',
  '/home/nfs/bzaitlen/GitRepos/dask-ml',
  '/home/nfs/bzaitlen/GitRepos/dask-cuda',
  '/home/nfs/bzaitlen/GitRepos/ucx-py',
  '/home/nfs/bzaitlen/GitRepos/pynvml',
  '/home/nfs/bzaitlen/GitRepos/cudf-ucx-fail-scripts'

In [30]:
from dummy import DummyColumn
d = DummyColumn()
h, f = d.serialize()
print(d.deserialize(h, f), f, type(f[0]))

[1 2 3] [array([    0,     1,     2, ..., 99997, 99998, 99999])] <class 'cupy.core.core.ndarray'>


In [14]:
worker_1, worker_2 = client.scheduler_info()['workers']

In [31]:
from dask.distributed import wait
import operator

left = client.map(lambda x: DummyColumn(), range(10000), workers=[worker_1])
right = client.map(lambda x: DummyColumn(), range(10000), workers=[worker_2])
results = client.map(lambda x,y: (x,y), left, right)
_ = wait(results)

In [ ]:
results[0].result()

In [28]:
import cudf
import numpy as np
import cupy

In [34]:
from dask.distributed import wait
import operator

left = client.map(lambda x: cupy.asarray(np.arange(100000)), range(10), workers=[worker_1])
right = client.map(lambda x: cupy.asarray(np.arange(100000)), range(10), workers=[worker_2])
results = client.map(lambda x,y: (x,y), left, right, priority=10)
_ = wait(results)

In [21]:
client

Client Scheduler: ucx://10.33.225.165:8986 Dashboard: http://10.33.225.165:8787/status,Cluster Workers: 2 Cores: 2 Memory: 64.00 GB


In [22]:
1+1

2